In [1]:
import os
import sys
import torch
import numpy as np
import pandas as pd 
from pathlib import Path

In [6]:
data_path_string = r"C:\Users\cesar\Desktop\Projects\BirthDeathPortafolioChoice\Codes\birth-death-portfolio\data\raw"
list_of_folders = os.listdir(data_path_string)

In [7]:
dates_and_files_number = []
for folder_string in list_of_folders:
    if "2021" in folder_string:
        path_to_data = Path(data_path_string) / folder_string
        number_of_files = len(os.listdir(path_to_data))
        dates_and_files_number.append((number_of_files,folder_string))
        print(f"Name: {folder_string}, Number of files {number_of_files}")

Name: 2021-05-10, Number of files 7076
Name: 2021-06-02, Number of files 1771
Name: 2021-06-08, Number of files 7683
Name: 2021-06-14, Number of files 7847
Name: 2021-06-15, Number of files 0
Name: 2021-06-16, Number of files 441
Name: 2021-06-17, Number of files 595
Name: 2021-06-18, Number of files 591
Name: 2021-06-19, Number of files 585
Name: 2021-06-20, Number of files 1004
Name: 2021-06-21, Number of files 593
Name: 2021-10-24, Number of files 8202
Name: 2021-10-25, Number of files 67
Name: 2021-10-26, Number of files 14


In [4]:
dates_and_files_number.sort()
dates_and_files_number

[(0, '2021-06-15'),
 (14, '2021-10-26'),
 (67, '2021-10-25'),
 (441, '2021-06-16'),
 (585, '2021-06-19'),
 (591, '2021-06-18'),
 (593, '2021-06-21'),
 (595, '2021-06-17'),
 (1004, '2021-06-20'),
 (1771, '2021-06-02'),
 (7076, '2021-05-10'),
 (7683, '2021-06-08'),
 (7847, '2021-06-14'),
 (8202, '2021-10-24')]

In [3]:
path_to_data_date = Path(data_path_string) / '2021-05-10'
coin_file_string_name = os.listdir(path_to_data_date)[0]
path_to_coin_data = path_to_data_date / coin_file_string_name

In [4]:
pd.read_csv(path_to_coin_data,nrows=2)

,Unnamed: 0,price,market_cap,volume
0,2021-10-24 04:52:29.596,22688.706092,0.0,6968.190273
1,2021-10-24 05:13:47.385,22690.000000,0.0,3421.912980


In [8]:
import pandas as pd
from tqdm import tqdm
import os

# Define the path to your folder containing the CSV files
folder_path = path_to_data_date

# Initialize a dictionary to store the summary for each file
files_summary = {}

# Function to update the summary for each column
def update_summary(df, col_name, summary_dict):
    summary_dict[col_name] = {
        'min_value': df[col_name].min(),
        'max_value': df[col_name].max(),
        'min_date': df.loc[df[col_name].idxmin(), 'date'],
        'max_date': df.loc[df[col_name].idxmax(), 'date']
    }

# Iterate over each file in the folder
for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith('.csv'):  # Check if the file is a CSV
        file_path = os.path.join(folder_path, filename)
        
        try:
            # Read the entire file
            df = pd.read_csv(file_path)
            df['date'] = pd.to_datetime(df.iloc[:, 0])  # Assuming the date is in the first column
            
            # Initialize summary for this file
            file_summary = {
                'num_rows': len(df),
            }
            
            # Update summary for each column, including the date column
            for col in df.columns:
                if col != df.columns[0]:  # Exclude the first column if it's just a datetime index
                    update_summary(df, col, file_summary)
            
            # Store the file summary in the main dictionary
            files_summary[filename] = file_summary
            
        except Exception as e:
            print(f"Error processing {filename}: {e}")

  0%|          | 17/8201 [00:00<00:49, 166.54it/s]

 39%|███▉      | 3206/8201 [00:10<00:18, 267.19it/s]C:\Users\cesar\AppData\Local\Temp\ipykernel_29368\1492377003.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df.iloc[:, 0])  # Assuming the date is in the first column
100%|██████████| 8201/8201 [00:29<00:00, 274.15it/s]


In [20]:
import os
import pandas as pd
from tqdm import tqdm

# Define the path to your folder containing the CSV files
folder_path = path_to_data_date  # Update this to your folder path
files_list = []

# Function to process each file and update the summary with detailed stats for each column
def process_file(file_path, filename):
    df = pd.read_csv(file_path)
    df['date'] = pd.to_datetime(df.iloc[:, 0])  # Assuming the date is in the first column
    
    file_summary = {'filename': filename}
    
    # Loop through each column, except the datetime index, to gather stats
    for col in df.columns[1:]:  # Skip the first column assumed to be datetime
        col_data = {
            f'{col}_min_value': df[col].min(),
            f'{col}_max_value': df[col].max(),
            f'{col}_min_date': df.loc[df[col].idxmin(), 'date'],
            f'{col}_max_date': df.loc[df[col].idxmax(), 'date']
        }
        file_summary.update(col_data)
    
    # Additional summary information
    file_summary['num_rows'] = len(df)
    
    return file_summary

# Iterate over each file in the folder and process it
for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith('.csv'):  # Ensure the file is a CSV
        file_path = os.path.join(folder_path, filename)
        try:
            file_info = process_file(file_path, filename)
            files_list.append(file_info)
        except Exception as e:
            print(f"Error processing {filename}: {e}")

# Create a DataFrame from the collected file summaries
summary_df = pd.DataFrame(files_list)

  0%|          | 0/8201 [00:00<?, ?it/s]

 39%|███▉      | 3200/8201 [00:10<00:15, 321.99it/s]C:\Users\cesar\AppData\Local\Temp\ipykernel_29368\861275737.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df.iloc[:, 0])  # Assuming the date is in the first column
100%|██████████| 8201/8201 [00:26<00:00, 313.72it/s]


In [21]:
summary_df.head()

,filename,price_min_value,price_max_value,price_min_date,price_max_date,market_cap_min_value,market_cap_max_value,market_cap_min_date,market_cap_max_date,volume_min_value,volume_max_value,volume_min_date,volume_max_date,date_min_value,date_max_value,date_min_date,date_max_date,num_rows
0,0-5x-long-algorand-token_day.csv,22688.706092,22691.000395,2021-10-24 04:52:29.596,2021-10-24 05:43:18.669,0.0,0.0,2021-10-24 04:52:29.596,2021-10-24 04:52:29.596,0.909380,6968.190273,2021-10-24 05:43:18.669,2021-10-24 04:52:29.596,2021-10-24 04:52:29.596,2021-10-24 05:43:18.669,2021-10-24 04:52:29.596,2021-10-24 05:43:18.669,4
1,0-5x-long-cardano-token_day.csv,57949.618126,57949.618126,2021-10-24 15:12:26.215,2021-10-24 15:12:26.215,0.0,0.0,2021-10-24 15:12:26.215,2021-10-24 15:12:26.215,94.226079,94.226079,2021-10-24 15:12:26.215,2021-10-24 15:12:26.215,2021-10-24 15:12:26.215,2021-10-24 15:12:26.215,2021-10-24 15:12:26.215,2021-10-24 15:12:26.215,1
2,0-5x-long-cosmos-token_day.csv,27467.400539,27724.000000,2021-10-23 16:35:06.273,2021-10-23 15:52:50.162,0.0,0.0,2021-10-23 15:52:50.162,2021-10-23 15:52:50.162,1.657270,20892.195880,2021-10-24 10:20:59.467,2021-10-23 16:23:54.852,2021-10-23 15:52:50.162,2021-10-24 10:20:59.467,2021-10-23 15:52:50.162,2021-10-24 10:20:59.467,7
3,0-5x-long-defi-index-token_day.csv,18240.828216,18240.828216,2021-10-24 13:47:27.252,2021-10-24 13:47:27.252,0.0,0.0,2021-10-24 13:47:27.252,2021-10-24 13:47:27.252,29.732550,29.732550,2021-10-24 13:47:27.252,2021-10-24 13:47:27.252,2021-10-24 13:47:27.252,2021-10-24 13:47:27.252,2021-10-24 13:47:27.252,2021-10-24 13:47:27.252,1
4,0-5x-long-dogecoin-token_day.csv,144100.000000,148969.875576,2021-10-24 21:48:52.525,2021-10-25 01:41:23.361,0.0,0.0,2021-10-24 21:48:52.525,2021-10-24 21:48:52.525,14.410000,29.304074,2021-10-24 21:48:52.525,2021-10-25 01:41:23.361,2021-10-24 21:48:52.525,2021-10-25 01:41:23.361,2021-10-24 21:48:52.525,2021-10-25 01:41:23.361,2


In [22]:
# Sort summary_df by 'market_cap_max_value' in descending order
sorted_summary_df = summary_df.sort_values(by='market_cap_max_value', ascending=False)

In [24]:
sorted_summary_df.head()

,filename,price_min_value,price_max_value,price_min_date,price_max_date,market_cap_min_value,market_cap_max_value,market_cap_min_date,market_cap_max_date,volume_min_value,volume_max_value,volume_min_date,volume_max_date,date_min_value,date_max_value,date_min_date,date_max_date,num_rows
1150,bitcoin_day.csv,60141.462454,61875.029714,2021-10-24 14:18:17.224,2021-10-23 17:29:02.534,1.133463e+12,1.164723e+12,2021-10-24 14:24:27.929,2021-10-23 18:24:46.840,2.573977e+10,3.785352e+10,2021-10-24 13:54:47.857,2021-10-23 15:19:49.349,2021-10-23 15:19:49.349,2021-10-24 15:14:49.801,2021-10-23 15:19:49.349,2021-10-24 15:14:49.801,288
2787,ethereum_day.csv,4025.282324,4199.632883,2021-10-24 14:20:05.072,2021-10-24 02:13:57.836,4.749766e+11,4.940470e+11,2021-10-24 14:24:38.677,2021-10-24 02:04:56.654,1.576220e+10,1.963348e+10,2021-10-24 00:14:08.223,2021-10-23 15:24:08.611,2021-10-23 15:24:08.611,2021-10-24 15:18:42.172,2021-10-23 15:24:08.611,2021-10-24 15:18:42.172,288
1049,binancecoin_day.csv,473.379014,488.980010,2021-10-24 14:19:28.368,2021-10-24 02:14:01.252,7.959401e+10,8.208503e+10,2021-10-24 14:23:58.878,2021-10-24 01:34:18.741,8.464143e+08,1.507096e+09,2021-10-24 03:48:40.239,2021-10-23 15:19:55.298,2021-10-23 15:19:55.298,2021-10-24 15:13:29.106,2021-10-23 15:19:55.298,2021-10-24 15:13:29.106,288
7037,tether_day.csv,0.996616,1.014384,2021-10-24 13:56:35.751,2021-10-24 11:21:50.422,7.017836e+10,7.111715e+10,2021-10-24 13:51:20.565,2021-10-24 11:21:50.422,5.198615e+10,6.189903e+10,2021-10-24 01:11:50.394,2021-10-23 15:21:59.320,2021-10-23 15:21:59.320,2021-10-24 15:11:50.384,2021-10-23 15:21:59.320,2021-10-24 15:11:50.384,286
1542,cardano_day.csv,2.117723,2.178698,2021-10-24 14:29:05.105,2021-10-24 01:34:54.934,6.792400e+10,6.980081e+10,2021-10-24 13:55:16.335,2021-10-24 02:09:58.097,1.116585e+09,1.773229e+09,2021-10-24 07:50:18.693,2021-10-23 15:20:23.553,2021-10-23 15:20:23.553,2021-10-24 15:15:21.171,2021-10-23 15:20:23.553,2021-10-24 15:15:21.171,288


In [25]:

def save_dataframe(df, filepath, format='csv'):
    """
    Saves a DataFrame to a file.
    
    Args:
    - df: Pandas DataFrame to save.
    - filepath: Path and name of the file to save the DataFrame.
    - format: Format to save the DataFrame in ('csv' or 'pickle'). Defaults to 'csv'.
    """
    if format == 'csv':
        df.to_csv(filepath, index=False)
    elif format == 'pickle':
        df.to_pickle(filepath)
    else:
        raise ValueError("Unsupported format. Use 'csv' or 'pickle'.")

def load_dataframe(filepath, format='csv'):
    """
    Loads a DataFrame from a file.
    
    Args:
    - filepath: Path and name of the file to load the DataFrame from.
    - format: Format of the file to load the DataFrame from ('csv' or 'pickle'). Defaults to 'csv'.
    
    Returns:
    - df: The loaded Pandas DataFrame.
    """
    if format == 'csv':
        return pd.read_csv(filepath)
    elif format == 'pickle':
        return pd.read_pickle(filepath)
    else:
        raise ValueError("Unsupported format. Use 'csv' or 'pickle'.")

In [29]:
df = 

In [30]:
df.head()

,filename,price_min_value,price_max_value,price_min_date,price_max_date,market_cap_min_value,market_cap_max_value,market_cap_min_date,market_cap_max_date,volume_min_value,volume_max_value,volume_min_date,volume_max_date,date_min_value,date_max_value,date_min_date,date_max_date,num_rows
0,bitcoin_day.csv,60141.462454,61875.029714,2021-10-24 14:18:17.224,2021-10-23 17:29:02.534,1.133463e+12,1.164723e+12,2021-10-24 14:24:27.929,2021-10-23 18:24:46.840,2.573977e+10,3.785352e+10,2021-10-24 13:54:47.857,2021-10-23 15:19:49.349,2021-10-23 15:19:49.349,2021-10-24 15:14:49.801,2021-10-23 15:19:49.349,2021-10-24 15:14:49.801,288
1,ethereum_day.csv,4025.282324,4199.632883,2021-10-24 14:20:05.072,2021-10-24 02:13:57.836,4.749766e+11,4.940470e+11,2021-10-24 14:24:38.677,2021-10-24 02:04:56.654,1.576220e+10,1.963348e+10,2021-10-24 00:14:08.223,2021-10-23 15:24:08.611,2021-10-23 15:24:08.611,2021-10-24 15:18:42.172,2021-10-23 15:24:08.611,2021-10-24 15:18:42.172,288
2,binancecoin_day.csv,473.379014,488.980010,2021-10-24 14:19:28.368,2021-10-24 02:14:01.252,7.959401e+10,8.208503e+10,2021-10-24 14:23:58.878,2021-10-24 01:34:18.741,8.464143e+08,1.507096e+09,2021-10-24 03:48:40.239,2021-10-23 15:19:55.298,2021-10-23 15:19:55.298,2021-10-24 15:13:29.106,2021-10-23 15:19:55.298,2021-10-24 15:13:29.106,288
3,tether_day.csv,0.996616,1.014384,2021-10-24 13:56:35.751,2021-10-24 11:21:50.422,7.017836e+10,7.111715e+10,2021-10-24 13:51:20.565,2021-10-24 11:21:50.422,5.198615e+10,6.189903e+10,2021-10-24 01:11:50.394,2021-10-23 15:21:59.320,2021-10-23 15:21:59.320,2021-10-24 15:11:50.384,2021-10-23 15:21:59.320,2021-10-24 15:11:50.384,286
4,cardano_day.csv,2.117723,2.178698,2021-10-24 14:29:05.105,2021-10-24 01:34:54.934,6.792400e+10,6.980081e+10,2021-10-24 13:55:16.335,2021-10-24 02:09:58.097,1.116585e+09,1.773229e+09,2021-10-24 07:50:18.693,2021-10-23 15:20:23.553,2021-10-23 15:20:23.553,2021-10-24 15:15:21.171,2021-10-23 15:20:23.553,2021-10-24 15:15:21.171,288
